In [2]:
# When importing cobrapy, "import cobra" gets you most of the functions you need
import cobra

In [3]:
# first let's load one of the existing models encoded as a json file
# Remember that python uses relative paths, just like you're operating from the commandline.
# Because this jupyter notebook is in psy_recon/bin, we go back one level ('..') to 'psy_recon' then
# enter ''/results' to access the model.
model = cobra.io.load_json_model('../results/reconstructions/PST_pao1_temp_biomass_v4.json')

In [3]:
# let's inspect the model. When we're in a notebook and we enter the model variable
# at the end of a cell, cobrapy will output some summary information about the model
model

Name,PST
Memory address,0x011b2e7048
Number of metabolites,1027
Number of reactions,920
Objective expression,-1.0*PAO1_Biomass_reverse_55fa3 + 1.0*PAO1_Biomass
Compartments,"c, e"


In [4]:
# First, let's just save the model as an SBML file. We've already loaded the model
# in the "model" variable, so we just have to pass this to the SBML writing function.
# Remember to pass the additional argument use_fbc_package=True to save in the correct format.
# The second argument is the location and name of the new SBML file we are saving.
# We'll use the same filename as the json file, but swap out the extension for '.sbml'.
cobra.io.write_sbml_model(model,filename='../results/reconstructions/PST_pao1_temp_biomass_v4.sbml',use_fbc_package=True)

In [5]:
# Even when we save the model, it's still loaded in python as the model variable. 
# However, because the model might not be functional yet, let's load a test model
# to experiment with manipulating media conditions and doing flux balance analysis (FBA)
import cobra.test
ecoli = cobra.test.create_test_model('textbook')

In [6]:
# This test model describes basic metabolism in E. coli. let's inspect the model.
ecoli

Name,e_coli_core
Memory address,0x011b473a58
Number of metabolites,72
Number of reactions,95
Objective expression,-1.0*Biomass_Ecoli_core_reverse_2cdba + 1.0*Biomass_Ecoli_core
Compartments,"cytosol, extracellular"


In [7]:
# Cobrapy can automatically parse the environmental conditions (usually). As an example of this,
# inspect ecoli.medium, which specifies the uptake rate of boundary reactions.
ecoli.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

In [8]:
# Where do these uptake rates come from in the model? To see, let's inspect the actual reactions
# that are in the medium
medium = ecoli.medium

# the keys in the medium dictionary are reaction id's in the model
for reaction_id in medium.keys():
    # what does the reaction look like?
    print(ecoli.reactions.get_by_id(reaction_id))

EX_co2_e: co2_e <=> 
EX_glc__D_e: glc__D_e <=> 
EX_h_e: h_e <=> 
EX_h2o_e: h2o_e <=> 
EX_nh4_e: nh4_e <=> 
EX_o2_e: o2_e <=> 
EX_pi_e: pi_e <=> 


In [9]:
# Based on this, each of the reactions in the medium converts an extracellular metabolite (e.g. they all
# have the '_e' suffix) to nothing. If we wanted uptake of the metabolite into the system, we would need
# NEGATIVE flux for the reaction (e.g. 'nothing' is used to generate the metabolite)

# Let's inspect the lower bound of each of these reactions to see if they meet our expectation
# based on the uptake rates that medium describes.
for reaction_id in medium.keys():
    # what does the reaction look like?
    print(reaction_id,ecoli.reactions.get_by_id(reaction_id).lower_bound)

EX_co2_e -1000.0
EX_glc__D_e -10.0
EX_h_e -1000.0
EX_h2o_e -1000.0
EX_nh4_e -1000.0
EX_o2_e -1000.0
EX_pi_e -1000.0


In [10]:
# As you can see, they are all negative, allowing reverse flux through the reaction and therefore generation
# of each metabolite.

# Now, let's see if our E. coli model is able to grow in this media condition.
# in the model summary when we first loaded ecoli, we saw that the objective function was set to the
# biomass reaction. When we 'optimize' the ecoli model, it will attempt to maximize activity, or flux,
# through this biomass reaction.

# Let's optimize and take a look at the standard summary output
ecoli.optimize()

,fluxes,reduced_costs
ACALD,0.000000,6.938894e-18
ACALDt,0.000000,0.000000e+00
ACKr,0.000000,1.040834e-17
ACONTa,6.007250,0.000000e+00
ACONTb,6.007250,1.387779e-17
...,...,...
TALA,1.496984,-1.387779e-17
THD2,0.000000,-2.546243e-03
TKT1,1.496984,-1.387779e-17
TKT2,1.181498,1.387779e-17


In [11]:
# As you can see, the model was able to produce 0.874 units of biomass (i.e. the value of the objective)

# we can also save the solution to a varible to inspect it further, rather than printing.
solution = ecoli.optimize()
solution

,fluxes,reduced_costs
ACALD,0.000000,6.938894e-18
ACALDt,0.000000,0.000000e+00
ACKr,0.000000,1.040834e-17
ACONTa,6.007250,0.000000e+00
ACONTb,6.007250,1.387779e-17
...,...,...
TALA,1.496984,-1.387779e-17
THD2,0.000000,-2.546243e-03
TKT1,1.496984,-1.387779e-17
TKT2,1.181498,1.387779e-17


In [12]:
# Here are some examples of inspecting the solution. These are the actual variables used to construct the
# solution summary that cobrapy prints.
print(solution.objective_value)
print(solution.to_frame())

0.8739215069684307
                       fluxes  reduced_costs
ACALD                0.000000   6.938894e-18
ACALDt               0.000000   0.000000e+00
ACKr                 0.000000   1.040834e-17
ACONTa               6.007250   0.000000e+00
ACONTb               6.007250   1.387779e-17
ACt2r                0.000000   1.301043e-18
ADK1                 0.000000  -0.000000e+00
AKGDH                5.064376   1.318390e-16
AKGt2r               0.000000   1.517883e-17
ALCD2x               0.000000   0.000000e+00
ATPM                 8.390000  -1.018497e-02
ATPS4r              45.514010   4.336809e-19
Biomass_Ecoli_core   0.873922   0.000000e+00
CO2t               -22.809833  -1.387779e-17
CS                   6.007250   3.469447e-17
CYTBD               43.598985  -8.673617e-19
D_LACt2              0.000000   1.301043e-18
ENO                 14.716140   0.000000e+00
ETOHt2r              0.000000   1.301043e-18
EX_ac_e              0.000000  -4.583237e-02
EX_acald_e           0.000000  -6.87

In [13]:
# Given that the lower bound of these boundary reactions determines the uptake rate, the most
# reliable way to modify the environmental conditions is to change the lower bound. Let's
# do this for one of the exchange reactions that was already active and see if the model
# can still grow.
ecoli.reactions.get_by_id('EX_glc__D_e').lower_bound = 0

ecoli.optimize()

cobra/util/solver.py:408 UserWarning: solver status is 'infeasible'


<Solution infeasible at 0x11b53e860>

In [14]:
# When we turn off glucose uptake, the model becomes 'infeasible', meaning that there aren't any network-wide
# states that allow mass balance constraints to be met. This is equivalent to having 0 activity through the
# biomass reaction, so it essentially means the organism can't grow.

# Let's modify some of the other exchange reactions to see if we can restore growth. Use the .boundary
# attribute to find reactions that might be exchange reactions.
ecoli.boundary

[<Reaction EX_ac_e at 0x11bac4128>,
 <Reaction EX_acald_e at 0x11bac4198>,
 <Reaction EX_akg_e at 0x11bac4278>,
 <Reaction EX_co2_e at 0x11bac4390>,
 <Reaction EX_etoh_e at 0x11bac44a8>,
 <Reaction EX_for_e at 0x11bac45c0>,
 <Reaction EX_fru_e at 0x11bac46d8>,
 <Reaction EX_fum_e at 0x11bac47f0>,
 <Reaction EX_glc__D_e at 0x11bac4908>,
 <Reaction EX_gln__L_e at 0x11bac49e8>,
 <Reaction EX_glu__L_e at 0x11bac4b00>,
 <Reaction EX_h_e at 0x11bac4c88>,
 <Reaction EX_h2o_e at 0x11bac4dd8>,
 <Reaction EX_lac__D_e at 0x11bad7b70>,
 <Reaction EX_mal__L_e at 0x11bbd6b38>,
 <Reaction EX_nh4_e at 0x11bc0d5f8>,
 <Reaction EX_o2_e at 0x11bac4f60>,
 <Reaction EX_pi_e at 0x11bbeb0f0>,
 <Reaction EX_pyr_e at 0x11bbeb208>,
 <Reaction EX_succ_e at 0x11bbeb2e8>]

In [15]:
# Let's enable uptake through one of these that wasn't in the original medium.
ecoli.reactions.get_by_id('EX_gln__L_e').lower_bound = -10

# Does this restore biomass production?
ecoli.optimize()

,fluxes,reduced_costs
ACALD,0.000000,1.387779e-17
ACALDt,0.000000,0.000000e+00
ACKr,0.000000,2.428613e-17
ACONTa,2.103400,0.000000e+00
ACONTb,2.103400,1.387779e-16
...,...,...
TALA,-0.100053,1.387779e-17
THD2,0.000000,-5.371868e-03
TKT1,-0.100053,-2.081668e-17
TKT2,-0.301947,-0.000000e+00


In [16]:
# It does! Although it appears to be growing at a slower rate than before (e.g. 0.559 vs. 0.874)
# We'll add on to this notebook later with some more examples.